# ТЕСТОВОЕ ЗАДАНИЕ НА ВАКАНСИЮ “Backend разработчик (Python, Junior)”

In [1]:
import requests as rq
import pandas as pd

In [3]:
path_to_data = "https://file.notion.so/f/s/0f8850ad-e46f-4f37-99ea-0e4e2a6af5b6/trial_task.json?id=2583a04b-4256-4c1f-939e-6eac0f749ceb&table=block&spaceId=41165294-a784-489a-a401-1a916d020564&expirationTimestamp=1689868800000&signature=-1D4yL8SjErQtHRmRLR8SDvsSk6n_DPQ5uefnI1oeLs&downloadName=trial_task.json"
response = rq.get(path_to_data)
response.status_code

200

In [6]:
data = response.json()
data

[{'order_id': 11973,
  'warehouse_name': 'Мордор',
  'highway_cost': -70,
  'products': [{'product': 'ломтик июльского неба',
    'price': 450,
    'quantity': 1},
   {'product': 'билет в Израиль', 'price': 1000, 'quantity': 3},
   {'product': 'статуэтка Ленина', 'price': 200, 'quantity': 3}]},
 {'order_id': 62239,
  'warehouse_name': 'хутор близ Диканьки',
  'highway_cost': -15,
  'products': [{'product': 'билет в Израиль', 'price': 1000, 'quantity': 1}]},
 {'order_id': 85794,
  'warehouse_name': 'отель Лето',
  'highway_cost': -50,
  'products': [{'product': 'зеленая пластинка', 'price': 10, 'quantity': 2}]},
 {'order_id': 33684,
  'warehouse_name': 'Мордор',
  'highway_cost': -30,
  'products': [{'product': 'билет в Израиль', 'price': 1000, 'quantity': 2},
   {'product': 'зеленая пластинка', 'price': 10, 'quantity': 1}]},
 {'order_id': 25824,
  'warehouse_name': 'отель Лето',
  'highway_cost': -75,
  'products': [{'product': 'автограф Стаса Барецкого',
    'price': 600,
    'quantit

In [134]:
df = pd.json_normalize(data, 'products', ['order_id', 'warehouse_name', 'highway_cost'])

reordered = ['order_id', 'product', 'price', 'quantity', 'warehouse_name', 'highway_cost']
df = df.reindex(columns=reordered)
df

,order_id,product,price,quantity,warehouse_name,highway_cost
0,11973,ломтик июльского неба,450,1,Мордор,-70
1,11973,билет в Израиль,1000,3,Мордор,-70
2,11973,статуэтка Ленина,200,3,Мордор,-70
3,62239,билет в Израиль,1000,1,хутор близ Диканьки,-15
4,85794,зеленая пластинка,10,2,отель Лето,-50
...,...,...,...,...,...,...
192,79293,автограф Стаса Барецкого,600,1,отель Лето,-75
193,79293,ломтик июльского неба,450,1,отель Лето,-75
194,2930,плюмбус,250,2,Мордор,-30
195,2930,плюмбус,250,1,Мордор,-30


## 1. Тариф стоимости доставки для каждого склада

In [135]:
grouped = df.groupby(['warehouse_name', 'order_id'])
tariffs = grouped.highway_cost.first() // grouped.quantity.sum()
tariffs = tariffs.groupby('warehouse_name').first()
tariffs

warehouse_name
Мордор                 -10
гиперборея             -20
остров невезения        -5
отель Лето             -25
хутор близ Диканьки    -15
dtype: object

In [138]:
tariffs['хутор близ Диканьки']

-15

## 2. Суммарное количество , суммарный доход , суммарный расход и суммарная прибыль для каждого товара

In [107]:
wh_grouped.first()

,order_id,product,price,quantity,highway_cost
warehouse_name,,,,,
Мордор,11973,ломтик июльского неба,450,1,-70
гиперборея,58598,плюмбус,250,3,-160
остров невезения,87044,плюмбус,250,3,-15
отель Лето,85794,зеленая пластинка,10,2,-50
хутор близ Диканьки,62239,билет в Израиль,1000,1,-15


In [115]:
quants = df.groupby(['warehouse_name', 'order_id']).quantity.sum()
quants

warehouse_name       order_id
Мордор               2930        3
                     11973       7
                     16240       2
                     26702       4
                     27443       1
                                ..
хутор близ Диканьки  84471       4
                     85787       6
                     98423       2
                     99220       5
                     99246       3
Name: quantity, Length: 100, dtype: int64

In [116]:
costs = df.groupby(['warehouse_name', 'order_id']).highway_cost.first()
costs

warehouse_name       order_id
Мордор               2930        -30
                     11973       -70
                     16240       -20
                     26702       -40
                     27443       -10
                                ... 
хутор близ Диканьки  84471       -60
                     85787       -90
                     98423       -30
                     99220       -75
                     99246       -45
Name: highway_cost, Length: 100, dtype: object

In [118]:
tariffs = costs / quants
tariffs

warehouse_name       order_id
Мордор               2930       -10.0
                     11973      -10.0
                     16240      -10.0
                     26702      -10.0
                     27443      -10.0
                                 ... 
хутор близ Диканьки  84471      -15.0
                     85787      -15.0
                     98423      -15.0
                     99220      -15.0
                     99246      -15.0
Length: 100, dtype: object

In [125]:
tariffs.groupby('warehouse_name').first()

warehouse_name
Мордор                -10.0
гиперборея            -20.0
остров невезения       -5.0
отель Лето            -25.0
хутор близ Диканьки   -15.0
dtype: object

In [40]:
wh_highway_cost = ['warehouse_name', 'highway_cost']
wh_rates = df[wh_highway_cost].groupby('warehouse_name').max()
wh_rates.rename(columns={'highway_cost': 'tariff'}, inplace=True)

wh_rates

,tariff
warehouse_name,
Мордор,-10
гиперборея,-20
остров невезения,-5
отель Лето,-25
хутор близ Диканьки,-15


In [44]:
wh_rates.loc['Мордор'].tariff

-10

In [ ]:
group = df.groupby('order_id')